# Image-to-Image translation with SnapML
### What are image-to-image models
Image-to-image translation is a class of vision and graphics problems where the goal is to learn the mapping between an input image and an output image. It can be applied to a wide range of applications, such as collection style transfer, object transfiguration, season transfer and photo enhancement. One of the best approaches to these tasks is to use Generative Adverseral Networks (GANs).

**Note**: It is recommended to upload and run this notebook in Google Colab. You may need to modify some code to suppress warnings/errors running locally as some functions depends on Colab only libraries. 


### Examples
Some popular Image-to-image architectures consistst of:
1. Pix2Pix (https://github.com/phillipi/pix2pix)

In [ ]:
from IPython.display import Image
Image(url='https://github.com/phillipi/pix2pix/raw/master/imgs/examples.jpg', width = 800)


2. CycleGAN (https://github.com/junyanz/CycleGAN)

In [ ]:
Image(url='https://github.com/junyanz/CycleGAN/raw/master/imgs/horse2zebra.gif')

3. BicycleGAN (https://github.com/junyanz/BicycleGAN)

In [ ]:
Image(url='https://github.com/junyanz/BicycleGAN/raw/master/imgs/day2night.gif')

### Pros and Cons of GANs
GANs or Conditional GANs have been successful ingenerating hig-fidelity images. However, thye often suffer from:


*   Tremendous computational costs
*   Bulky memory usage

ALl these makes running the original Image-to-Image models almost impossible on mobile devices






### GAN Compression techiniques and CAT
In order to make GANs smaller while preserving the quality, there are many successful GAN compression techniques:

*   **GAN Compression** - Efficient Architectures for Interactive Conditional GANs  [[paper](https://arxiv.org/pdf/2003.08936.pdf)]
*   **GAN-slimming** - All-in-One GAN Compression by A Unified Optimization Framework  [[paper](https://arxiv.org/pdf/2008.11062.pdf)]

And in this notebook, we are going to demonstrate how to use a novel techinque where a teacher model can both provide a search space and perform distillation for student network [[paper](https://arxiv.org/pdf/2103.03467.pdf)] [[code](https://github.com/snap-research/CAT)]

With this techniques, we are able to run distilled generative models on most mobile devices at real time while preserving the same image quality.
Please see below a sample output for transforming images to  [Ukiyo-e](https://en.wikipedia.org/wiki/Ukiyo-e) paintings


In [9]:
from IPython.core.display import display, HTML
urls = ['https://raw.githubusercontent.com/snap-research/CAT/tutorial/demo/original.gif',
        'https://raw.githubusercontent.com/snap-research/CAT/tutorial/demo/ukiyoe.gif']

def make_html(image):
     return '<img src="{}" style="display:inline;margin:1px"/>'.format(image)

display(HTML(''.join(make_html(url) for url in urls)))

### Pre-requisites
*   Pretrained teacher (original) model 
*   Pretrained discriminator model 
*   Dataset statistics for FID computation

We provide the pre-requisites for ukiyoe2photo.


### Get training code

In [1]:
# install dependencies
%pip install tensorboardX onnx
!pip install torch==1.7 torchvision==0.8.0 torchaudio==0.7.0

!git clone https://github.com/snap-research/CAT.git
%cd CAT
!git fetch 
!git checkout tutorial

     |████████████████████████████████| 124 kB 5.1 MB/s 
     |████████████████████████████████| 12.7 MB 42.1 MB/s 
     |████████████████████████████████| 776.7 MB 4.2 kB/s 
     |████████████████████████████████| 11.8 MB 31.9 MB/s 
     |████████████████████████████████| 7.6 MB 30.7 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.10.0+cu111
    Uninstalling torchaudio-0.10.0+cu111:
      Successfully uninstalled torchaudio-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depe

### Download dataset
Download one of the public ukiyoe2photo datasets.


Or use your own dataset by creating the appropriate folders and adding in the images. Follow the instructions [here](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/docs/datasets.md#pix2pix-datasets).

In [ ]:
# example for ukiyoe2photo dataset on CycleGAN
# properties: Used on CycleGAN 
#             Unpaired dataset 
import os

FILE = 'ukiyoe2photo' 

BASE_DATA_DIR = './database/'
FILE_ZIP = FILE+'.zip'
URL = 'https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/'+FILE_ZIP 
TARGET_DIR = os.path.join(BASE_DATA_DIR, FILE)

!mkdir -p $BASE_DATA_DIR
!wget -N $URL -O $FILE_ZIP
!mkdir -p $TARGET_DIR
!unzip -q $FILE_ZIP -d $BASE_DATA_DIR 
!rm $FILE_ZIP

for details.

--2022-01-12 03:18:20--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/ukiyoe2photo.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.244.190
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 292946532 (279M) [application/zip]
Saving to: ‘ukiyoe2photo.zip’

ukiyoe2photo.zip    100%[===================>] 279.38M  2.38MB/s    in 3m 24s  

2022-01-12 03:21:44 (1.37 MB/s) - ‘ukiyoe2photo.zip’ saved [292946532/292946532]



In [ ]:
# if valA , valB don't exist 
# copy or create a softlink as you wish 

!cp -r $TARGET_DIR/testA/  $TARGET_DIR/valA
!cp -r $TARGET_DIR/testB/ $TARGET_DIR/valB

### Download pre-trained teacher models 

Download our pre-trained teacher models (generator and discriminator) and the calculated statistics used for FID.


In [ ]:
# download ukyio teacher generator
!gdown --id 1_VrjIowjfj-CxA5LX-P5vMcDY8pBPwu_
# download ukyio teacher discriminator
!gdown --id 1dM_qHcyx_XTBRTsSg82HMgUAHw8zElA4
# download ukyio real stats
!gdown --id 1LfFto-IhmpjRktkGlAETfiikVUpabxmv

Downloading...
From: https://drive.google.com/uc?id=1_VrjIowjfj-CxA5LX-P5vMcDY8pBPwu_
To: /content/CAT/ukiyo_teacher_iter68000_net_G_B.pth
100% 32.8M/32.8M [00:00<00:00, 124MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1dM_qHcyx_XTBRTsSg82HMgUAHw8zElA4
To: /content/CAT/ukiyo_teacher_iter68000_net_D_B.pth
100% 22.2M/22.2M [00:00<00:00, 135MB/s]
Downloading...
From: https://drive.google.com/uc?id=1LfFto-IhmpjRktkGlAETfiikVUpabxmv
To: /content/CAT/ukiyo_A.npz
100% 33.6M/33.6M [00:00<00:00, 127MB/s] 


## Training NPU-Friendly student model

In [ ]:
# The teacher model is trained with instance normalization and reflect padding
# For NPU-Friendly student model, we prune the teacher model to the target FLOPs, 
# and replace the padding to zero padding, normalization to batch norm w/ 
# tracking running stats.

NUM_EPOCHS = 50 

!python distill.py --dataroot ./database/ukiyoe2photo \
--log_dir logs/cycle_gan/ukiyoe2photo/inception/student/2p6B \
--restore_teacher_G_path ukiyo_teacher_iter68000_net_G_B.pth \
--restore_pretrained_G_path ukiyo_teacher_iter68000_net_G_B.pth \
--restore_D_path ukiyo_teacher_iter68000_net_D_B.pth \
--real_stat_path ukiyo_A.npz \
--dataset_mode unaligned \
--distiller inception \
--gan_mode lsgan \
--nepochs $NUM_EPOCHS --nepochs_decay $NUM_EPOCHS \
--teacher_netG inception_9blocks --student_netG inception_9blocks \
--pretrained_ngf 64 --teacher_ngf 64 --student_ngf 20 \
--ndf 64 \
--num_threads 2 \
--eval_batch_size 2 \
--batch_size 10 \
--gpu_ids 0 \
--norm_affine \
--norm_affine_D \
--channels_reduction_factor 6 \
--kernel_sizes 1 3 5 \
--lambda_distill 2.8 \
--lambda_recon 1000 \
--prune_cin_lb 16 \
--target_flops 2.6e9 \
--distill_G_loss_type ka \
--netD multi_scale \
--crop_size 512,256 \
--preprocess resize_and_crop \
--load_size 600 \
--save_epoch_freq 1 \
--save_latest_freq 500 \
--direction BtoA \
--norm_student batch \
--padding_type_student zero \
--norm_affine_student \
--norm_track_running_stats_student

----------------- Options ---------------
                active_fn: nn.ReLU                       
              active_fn_D: nn.LeakyReLU                  
             aspect_ratio: 1.0                           
               batch_size: 10                            	[default: 1]
                    beta1: 0.5                           
                 channels: None                          
channels_reduction_factor: 6                             	[default: 1]
          cityscapes_path: database/cityscapes-origin    
                crop_size: 512,256                       	[default: 256, 256]
                 dataroot: ./database/ukiyoe2photo       	[default: None]
             dataset_mode: unaligned                     	[default: aligned]
                direction: BtoA                          	[default: AtoB]
          display_winsize: 256                           
      distill_G_loss_type: ka                            	[default: mse]
                distiller: incepti

### Exporting model to ONNX

In [ ]:
%pip install torchprofile

!python onnx_export.py --dataroot ./database/ukiyoe2photo \
--log_dir onnx_files/cycle_gan/ukiyoe2photo/inception/student/2p6B \
--restore_teacher_G_path ukiyo_teacher_iter68000_net_G_B.pth \
--pretrained_student_G_path logs/cycle_gan/ukiyoe2photo/inception/student/2p6B/checkpoints/2_net_G.pth \
--real_stat_path ukiyo_A.npz \
--dataset_mode unaligned \
--pretrained_ngf 64 --teacher_ngf 64 --student_ngf 20 \
--gpu_ids 0 \
--norm_affine \
--channels_reduction_factor 6 \
--kernel_sizes 1 3 5 \
--prune_cin_lb 16 \
--target_flops 2.6e9 \
--load_size 600 \
--crop_size 512,256 \
--norm_student batch \
--padding_type_student zero \
--norm_affine_student \
--norm_track_running_stats_student \

----------------- Options ---------------
                active_fn: nn.ReLU                       
              active_fn_D: nn.LeakyReLU                  
             aspect_ratio: 1.0                           
               batch_size: 1                             
                    beta1: 0.5                           
                 channels: None                          
channels_reduction_factor: 6                             	[default: 1]
          cityscapes_path: database/cityscapes-origin    
                crop_size: 512,256                       	[default: 256, 256]
                 dataroot: ./database/ukiyoe2photo       	[default: None]
             dataset_mode: unaligned                     	[default: aligned]
                direction: AtoB                          
          display_winsize: 256                           
      distill_G_loss_type: mse                           
                distiller: inception                     
                 drn

### Add Preprocssing and Postprocessing to ONNX model

By default, our model is trained with inputs and outputs in the range of [-1,1]. However when importing into Lens Studio, we expect our inputs/outputs consists of RGB values in the range of [0, 255]. Here, we demonstrates how we can alter our ONNX model to normalize our inputs and scale outputs back for Lens Studio import.

In [13]:
import onnx
import compose # compose.py
from onnx import helper, checker

def make_linear_model(alpha, beta, input_shape, graph_name, input_name, output_name,):
    inputs = [helper.make_tensor_value_info(input_name, onnx.TensorProto.FLOAT, shape=input_shape)]
    outputs = [helper.make_tensor_value_info(output_name, onnx.TensorProto.FLOAT, shape=input_shape)]
    
    # X = X * alpha
    scale_name = f"{graph_name}:scale"
    scale_output_name = f"{graph_name}:scaled"
    scale_tensor = helper.make_tensor(scale_name, onnx.TensorProto.FLOAT, [1], [alpha])
    scale = helper.make_node('Constant', [], [scale_name], scale_name, value=scale_tensor)
    mul = helper.make_node('Mul',inputs=[input_name, scale_name], outputs=[scale_output_name], name=scale_output_name)
    
    # X = X + beta
    bias_name = f"{graph_name}:shift"
    bias_output_name = f"{graph_name}:shifted"
    bias_tensor = helper.make_tensor(bias_name, onnx.TensorProto.FLOAT, [1], [beta])
    bias = helper.make_node('Constant', [], [bias_name], bias_name, value=bias_tensor)
    add = helper.make_node('Add', inputs=[scale_output_name, bias_name], outputs=[output_name], name=bias_output_name)
    
    # Create graph and model
    graph = helper.make_graph([scale, mul, bias, add], graph_name, inputs, outputs)
    model = helper.make_model(graph, producer_name='snapml-demo')
    
    # align with 2p6BnetG_student's IR and Opset version
    model.ir_version = 6
    model.opset_import[0].version = 11

    # validate model
    checker.check_model(model)
    return model

def scale_model_for_studio(original_model_path, new_model_path):
    original_model = onnx.load(original_model_path) 
    input_dim = original_model.graph.input[0].type.tensor_type.shape.dim
    input_shape = ['None'] + [item.dim_value for item in input_dim[1:]]

    # add preprocessing to normalize inputs from [0,255] to [-1,1]
    preprocessing_model = make_linear_model(2/255, -1, input_shape, 'preprocessing', 'raw_input', 'input')
    combined_model = compose.merge_models(
        preprocessing_model, original_model,
        io_map=[('input', 'input')]
    )

    # add postprocessing to transform outputs back to [0,255]
    postprocessing_model = make_linear_model(255/2, 255/2, input_shape, 'postprocessing', 'output', 'raw_output')
    combined_model = compose.merge_models(
        combined_model, postprocessing_model,
        io_map=[('output', 'output')]
    )
    
    # validate model
    checker.check_model(combined_model)

    # save scaled model
    print(f"Saving scaled model to {new_model_path}")
    onnx.save(combined_model, new_model_path)

### Download onnx file to local

In [12]:
def download_onnx_from_colab(path):
    from google.colab import files
    files.download(path)

ONNX_PATH = 'onnx_files/cycle_gan/ukiyoe2photo/inception/student/2p6BnetG_student.onnx'
SCALED_ONNX_PATH = ONNX_PATH.replace('.onnx', '_scaled.onnx')
scale_model_for_studio(ONNX_PATH, SCALED_ONNX_PATH)
download_onnx_from_colab(SCALED_ONNX_PATH)

### Pre-trained student model

We also provide pre-trained student models

In [16]:
# download 2.6B FLOPs ukyio student onnx model
!gdown --id 1hxQi5N9Ft8nhMm5ckVFQvlWm2zWrOUad
# download to local
ONNX_PATH = 'ukyio-2p6BnetG_student.onnx'
SCALED_ONNX_PATH = ONNX_PATH.replace('.onnx', '_scaled.onnx')
scale_model_for_studio(ONNX_PATH, SCALED_ONNX_PATH)
download_onnx_from_colab(SCALED_ONNX_PATH)

Downloading...
From: https://drive.google.com/uc?id=1hxQi5N9Ft8nhMm5ckVFQvlWm2zWrOUad
To: /content/CAT/ukyio-2p6BnetG_student.onnx
100% 667k/667k [00:00<00:00, 95.6MB/s]
Saving scaled model to ukyio-2p6BnetG_student_scaled.onnx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
# download 5.2B FLOPs ukyio student onnx model
!gdown --id 1L4iq5KrhtrUZ9tYzD-4pOl_nl1k4cOwt
# download to local
ONNX_PATH = 'ukyio-5p2BnetG_student.onnx'
SCALED_ONNX_PATH = ONNX_PATH.replace('.onnx', '_scaled.onnx')
scale_model_for_studio(ONNX_PATH, SCALED_ONNX_PATH)
download_onnx_from_colab(SCALED_ONNX_PATH)

Downloading...
From: https://drive.google.com/uc?id=1L4iq5KrhtrUZ9tYzD-4pOl_nl1k4cOwt
To: /content/CAT/ukyio-5p2BnetG_student.onnx
100% 1.72M/1.72M [00:00<00:00, 115MB/s]
Saving scaled model to ukyio-5p2BnetG_student_scaled.onnx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>